In [72]:
import pandas as pd
import numpy as np

In [73]:
df=pd.read_csv('Big_Black_Money_Dataset.csv')
df.head()

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country
0,TX0000000001,Brazil,3.267530e+06,Offshore Transfer,2013-01-01 00:00:00,Person_1101,Construction,USA,True,Illegal,6,1,Bank_40,Singapore
1,TX0000000002,China,4.965767e+06,Stocks Transfer,2013-01-01 01:00:00,Person_7484,Luxury Goods,South Africa,False,Illegal,9,0,Bank_461,Bahamas
2,TX0000000003,UK,9.416750e+04,Stocks Transfer,2013-01-01 02:00:00,Person_3655,Construction,Switzerland,True,Illegal,1,3,Bank_387,Switzerland
3,TX0000000004,UAE,3.864201e+05,Cash Withdrawal,2013-01-01 03:00:00,Person_3226,Oil & Gas,Russia,False,Illegal,7,2,Bank_353,Panama
4,TX0000000005,South Africa,6.433784e+05,Cryptocurrency,2013-01-01 04:00:00,Person_7975,Real Estate,USA,True,Illegal,1,9,Bank_57,Luxembourg


In [74]:
#df.info()

In [75]:
df.isnull().sum()

Transaction ID                 0
Country                        0
Amount (USD)                   0
Transaction Type               0
Date of Transaction            0
Person Involved                0
Industry                       0
Destination Country            0
Reported by Authority          0
Source of Money                0
Money Laundering Risk Score    0
Shell Companies Involved       0
Financial Institution          0
Tax Haven Country              0
dtype: int64

In [76]:
#find any missing values and handle them
missing_value_percentage =(df.isnull().sum()/ len(df)) * 100
total_missing=df.isnull().sum()
missing_value_outcome=pd.concat([total_missing,missing_value_percentage],axis = 1)
missing_value_outcome

,0,1
Transaction ID,0,0.0
Country,0,0.0
Amount (USD),0,0.0
Transaction Type,0,0.0
Date of Transaction,0,0.0
Person Involved,0,0.0
Industry,0,0.0
Destination Country,0,0.0
Reported by Authority,0,0.0
Source of Money,0,0.0


In [77]:
#2 perform a sumary of statistics(mean.meadian.max) identify any xtreme outliers using z scores and IQR
df['Amount (USD)'].describe()

count    1.000000e+04
mean     2.501818e+06
std      1.424364e+06
min      1.003180e+04
25%      1.279005e+06
50%      2.501310e+06
75%      3.722416e+06
max      4.999812e+06
Name: Amount (USD), dtype: float64

In [78]:
#identify extreme outliers using z-scores
from scipy import stats
z_scores=stats.zscore(df['Amount (USD)'])
outliers=df[(z_scores > 3) | (z_scores < -3)]
print(outliers)

Empty DataFrame
Columns: [Transaction ID, Country, Amount (USD), Transaction Type, Date of Transaction, Person Involved, Industry, Destination Country, Reported by Authority, Source of Money, Money Laundering Risk Score, Shell Companies Involved, Financial Institution, Tax Haven Country]
Index: []


In [79]:
#identify extreme outliers using z-scores IQR-INTER QUAntile RANGE METHOD
Q1 =df['Amount (USD)'].quantile(0.25)
Q3 =df['Amount (USD)'].quantile(0.75)
IQR=Q3-Q1

lower_bound= Q1-1.5 *IQR
upper_bound= Q3+1.5*IQR
outlier_r=df[(df['Amount (USD)'] < lower_bound)|(df['Amount (USD)']>upper_bound)]
print(outlier_r)

Empty DataFrame
Columns: [Transaction ID, Country, Amount (USD), Transaction Type, Date of Transaction, Person Involved, Industry, Destination Country, Reported by Authority, Source of Money, Money Laundering Risk Score, Shell Companies Involved, Financial Institution, Tax Haven Country]
Index: []


summary

In [80]:
#Z-score method:detects outliers more than 3 standard deviation away fromthe mean
#IQR method:flags values outside the range of 1.5 *IQR from Q1 OR Q3

In [81]:
#HOW MANY UNIQUE VALUES ARE PRESENT IN THE COUNTRY AND DESTINATION COUNTRY COLUMNS,ARE THERE ANY DISCREPANCIES EG SPELLING INCONSISTANCES
COL=df[['Country','Destination Country']]
for x in COL:
    print(df[x].unique())

['Brazil' 'China' 'UK' 'UAE' 'South Africa' 'Russia' 'Switzerland' 'India'
 'USA' 'Singapore']
['USA' 'South Africa' 'Switzerland' 'Russia' 'Brazil' 'UK' 'India' 'China'
 'Singapore' 'UAE']


In [82]:
#convert the date of transaction column into components like year,month and day
#what are the frequent(most) transaction year and month
#1.covert object from object to datetime

In [127]:
df['Date of Transaction'] = pd.to_datetime(df['Date of Transaction'])
df['Year_of_Transaction']=pd.DatetimeIndex(df['Date of Transaction']).year
df['Month_of_Transaction']=pd.DatetimeIndex(df['Date of Transaction']).month
df['Day_of_Transaction']=pd.DatetimeIndex(df['Date of Transaction']).day
df.head()
df['Date of Transaction']

0      2013-01-10 10:00:00
1      2013-08-09 03:00:00
2      2014-02-20 11:00:00
3      2013-08-09 01:00:00
4      2013-08-20 06:00:00
               ...        
9995   2013-06-17 05:00:00
9996   2014-02-20 23:00:00
9997   2013-05-05 21:00:00
9998   2013-07-14 16:00:00
9999   2013-12-30 17:00:00
Name: Date of Transaction, Length: 10000, dtype: datetime64[ns]

In [84]:
#year and month with the highiest transactions
df['Year_of_Transaction'].value_counts().sort_values(ascending=False)

Year_of_Transaction
2013    8760
2014    1240
Name: count, dtype: int64

In [85]:
df['Month_of_Transaction'].value_counts().sort_values(ascending=False)

Month_of_Transaction
1     1488
2     1168
3      744
5      744
7      744
8      744
10     744
12     744
4      720
6      720
9      720
11     720
Name: count, dtype: int64

In [86]:
#perform frequency counts of categorical columns such as  transaCTION TYPE AND INDUSTRY
#ARE THERE ANY VALUES THAT OCCUR USUALLY OFTEN OR RARELY
transaction_type_counts = df['Transaction Type'].value_counts()
industry_counts= df['Industry'].value_counts()
print(transaction_type_counts)
print(industry_counts)

Transaction Type
Property Purchase    2086
Stocks Transfer      1983
Offshore Transfer    1980
Cash Withdrawal      1978
Cryptocurrency       1973
Name: count, dtype: int64
Industry
Finance         1475
Construction    1460
Luxury Goods    1459
Real Estate     1443
Arms Trade      1414
Casinos         1377
Oil & Gas       1372
Name: count, dtype: int64


In [87]:
#are there any values that occur unusally ofen or rarely
#first find the treshold range
transaction_type_count_per =df['Transaction Type'].value_counts()/ df['Transaction Type'].value_counts().sum()

In [88]:
#secondly define a threshold
rarely_threshold= 0.02*len(df)
often_threshold=  0.18 *len(df)
print(transaction_type_count_per)
print(transaction_type_counts[transaction_type_counts <rarely_threshold])
print(industry_counts[industry_counts <rarely_threshold])

Transaction Type
Property Purchase    0.2086
Stocks Transfer      0.1983
Offshore Transfer    0.1980
Cash Withdrawal      0.1978
Cryptocurrency       0.1973
Name: count, dtype: float64
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)


In [89]:
print(transaction_type_counts[transaction_type_counts <often_threshold])
print(industry_counts[industry_counts <often_threshold])

Series([], Name: count, dtype: int64)
Industry
Finance         1475
Construction    1460
Luxury Goods    1459
Real Estate     1443
Arms Trade      1414
Casinos         1377
Oil & Gas       1372
Name: count, dtype: int64


In [90]:
#after vs code installation
#from extenstion insatll
#!pylance
#python
#jupyter
#using windows power shell install 
#pip install ipykernel,numpy,pandas,seaborn,matplotlib,plotly,xlsxwriter,lxml,scikit-learn,nltk,wheel
#wordcloud,nbformat>=4.2.0,textblob,pymysql

In [91]:
###CREATE A NEW COLUMN TO CATEGORIZE AMOUNT (USD) INTO BINS (eg low,medium,high)how many transactions fall into each category

In [128]:
#check the max and min first
df['Amount (USD)'].max() - df['Amount (USD)'].min()
bins = [10031.80,500000,2000000,4999812.41]
labels=['LOW','MEDIM','HIGH']
df['AMOUNT CATEGORY']=pd.cut(df['Amount (USD)'],bins=bins,labels=labels,include_lowest=True)
df.head()


,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,...,Shell Companies Involved,Financial Institution,Tax Haven Country,Year_of_Transaction,Month_of_Transaction,Day_of_Transaction,AMOUNT CATEGORY,Time Diff,RISK INDEX,High Risk Flag
0,TX0000000227,Singapore,3.746944e+06,Offshore Transfer,2013-01-10 10:00:00,Person_1,Real Estate,USA,False,Legal,...,5,Bank_200,Luxembourg,2013,1,10,HIGH,NaT,14,1
1,TX0000005284,UAE,2.200251e+06,Cryptocurrency,2013-08-09 03:00:00,Person_10,Real Estate,India,False,Legal,...,0,Bank_77,Switzerland,2013,8,9,HIGH,NaT,9,0
2,TX0000009972,India,3.217272e+06,Property Purchase,2014-02-20 11:00:00,Person_100,Finance,UAE,False,Illegal,...,5,Bank_264,Switzerland,2014,2,20,HIGH,NaT,6,0
3,TX0000005282,USA,5.342882e+05,Cash Withdrawal,2013-08-09 01:00:00,Person_1000,Oil & Gas,UK,False,Illegal,...,9,Bank_349,Panama,2013,8,9,MEDIM,NaT,14,0
4,TX0000005551,Singapore,2.169575e+06,Stocks Transfer,2013-08-20 06:00:00,Person_1002,Luxury Goods,USA,False,Illegal,...,7,Bank_440,Cayman Islands,2013,8,20,HIGH,NaT,13,0


In [93]:
#how many transactions fall into each category
df['AMOUNT CATEGORY'].value_counts()

AMOUNT CATEGORY
HIGH     6010
MEDIM    3049
LOW       940
Name: count, dtype: int64

In [94]:
#cal the total number of transactions for each person involved
#what are the discriptive statistics(mean,median,max) for transaction count per person
transaction_count=df.groupby('Person Involved').size()
mean_transaction = transaction_count.mean()
median_transaction = transaction_count.median()
max_transaction = transaction_count.max()
print(f'the mean transaction for person involved is {mean_transaction}')
print(f'the median transaction for person involved is {median_transaction}')
print(f'the max transaction for person involved is {max_transaction}')

the mean transaction for person involved is 1.5822784810126582
the median transaction for person involved is 1.0
the max transaction for person involved is 7


In [95]:
df['Person Involved'].value_counts()

Person Involved
Person_7230    7
Person_6424    7
Person_1649    6
Person_3310    6
Person_2625    6
              ..
Person_6098    1
Person_9692    1
Person_1744    1
Person_4703    1
Person_3267    1
Name: count, Length: 6320, dtype: int64

In [96]:
#create a feature representing the transaction time difference between consecutive transaction for each person.what are the average mean and median time difference
df.sort_values(by=['Person Involved','Date of Transaction'],inplace=True,ignore_index=True)
df['Time Diff']=df.groupby('Person Involved')['Date of Transaction'].diff()
df.head()

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country,Year_of_Transaction,Month_of_Transaction,Day_of_Transaction,AMOUNT CATEGORY,Time Diff
0,TX0000000227,Singapore,3.746944e+06,Offshore Transfer,2013-01-10 10:00:00,Person_1,Real Estate,USA,False,Legal,9,5,Bank_200,Luxembourg,2013,1,10,HIGH,NaT
1,TX0000005284,UAE,2.200251e+06,Cryptocurrency,2013-08-09 03:00:00,Person_10,Real Estate,India,False,Legal,9,0,Bank_77,Switzerland,2013,8,9,HIGH,NaT
2,TX0000009972,India,3.217272e+06,Property Purchase,2014-02-20 11:00:00,Person_100,Finance,UAE,False,Illegal,1,5,Bank_264,Switzerland,2014,2,20,HIGH,NaT
3,TX0000005282,USA,5.342882e+05,Cash Withdrawal,2013-08-09 01:00:00,Person_1000,Oil & Gas,UK,False,Illegal,5,9,Bank_349,Panama,2013,8,9,MEDIM,NaT
4,TX0000005551,Singapore,2.169575e+06,Stocks Transfer,2013-08-20 06:00:00,Person_1002,Luxury Goods,USA,False,Illegal,6,7,Bank_440,Cayman Islands,2013,8,20,HIGH,NaT


In [97]:
df.tail()

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country,Year_of_Transaction,Month_of_Transaction,Day_of_Transaction,AMOUNT CATEGORY,Time Diff
9995,TX0000004014,Singapore,2.600608e+06,Cash Withdrawal,2013-06-17 05:00:00,Person_9995,Luxury Goods,China,False,Illegal,8,0,Bank_29,Singapore,2013,6,17,HIGH,108 days 18:00:00
9996,TX0000009984,USA,4.105025e+06,Cryptocurrency,2014-02-20 23:00:00,Person_9995,Construction,Brazil,True,Legal,5,5,Bank_284,Bahamas,2014,2,20,HIGH,248 days 18:00:00
9997,TX0000002998,Switzerland,2.727573e+06,Offshore Transfer,2013-05-05 21:00:00,Person_9998,Luxury Goods,China,False,Illegal,2,3,Bank_254,Singapore,2013,5,5,HIGH,NaT
9998,TX0000004673,UK,9.596637e+05,Stocks Transfer,2013-07-14 16:00:00,Person_9998,Casinos,UK,True,Illegal,2,7,Bank_450,Luxembourg,2013,7,14,MEDIM,69 days 19:00:00
9999,TX0000008730,India,2.647788e+06,Cash Withdrawal,2013-12-30 17:00:00,Person_9999,Luxury Goods,China,True,Illegal,7,8,Bank_220,Cayman Islands,2013,12,30,HIGH,NaT


In [98]:
#how many transactions involve tax haven countries,cal the proportion
df['Tax Haven Country'].value_counts().sort_values(ascending=False)

Tax Haven Country
Panama            1743
Luxembourg        1681
Cayman Islands    1676
Singapore         1644
Switzerland       1628
Bahamas           1628
Name: count, dtype: int64

In [99]:
df['Tax Haven Country'].value_counts().sort_values(ascending=False)/df ['Tax Haven Country'].value_counts().sort_values(ascending=False).sum()

Tax Haven Country
Panama            0.1743
Luxembourg        0.1681
Cayman Islands    0.1676
Singapore         0.1644
Switzerland       0.1628
Bahamas           0.1628
Name: count, dtype: float64

In [100]:
(df['Tax Haven Country'].value_counts().sort_values(ascending=False)/df ['Tax Haven Country'].value_counts().sort_values(ascending=False).sum())*100

Tax Haven Country
Panama            17.43
Luxembourg        16.81
Cayman Islands    16.76
Singapore         16.44
Switzerland       16.28
Bahamas           16.28
Name: count, dtype: float64

In [101]:
#create a new feature called RISK INDEX that combines money laundering risk score and the number of shell companies involved
#what is the range of and the average of this index
df['RISK INDEX']=df['Money Laundering Risk Score']+df['Shell Companies Involved']
print(f'mean risk index is {df['RISK INDEX'].mean()}')
print(f'range risk index is {df['RISK INDEX'].max()-df['RISK INDEX'].min()}')

mean risk index is 9.9958
range risk index is 18


In [102]:
#Calculate the average Money Laundaring risk score for each industry
#what are he industries nwith the highiest average risk
df.groupby('Industry')['Money Laundering Risk Score'].mean()

Industry
Arms Trade      5.570721
Casinos         5.556282
Construction    5.583562
Finance         5.711864
Luxury Goods    5.373544
Oil & Gas       5.427843
Real Estate     5.455301
Name: Money Laundering Risk Score, dtype: float64

In [103]:
df.groupby('Industry')['Money Laundering Risk Score'].mean().sort_values(ascending=False).head(4)

Industry
Finance         5.711864
Construction    5.583562
Arms Trade      5.570721
Casinos         5.556282
Name: Money Laundering Risk Score, dtype: float64

In [104]:
#create a binary flag to indictate high risk transactions where the money laundaring risk score is greater than 7 and more than 2 shel companies are incolved.how many transactions meet this criteria
df['High Risk Flag'] = ((df['Money Laundering Risk Score']>7) & (df['Shell Companies Involved']>2)).astype(int)
df['High Risk Flag'].sum()

np.int64(2115)

In [105]:
#what are te most common combinations of transaction type and industry? provide the top 5 most frequent transactions
common_combinations=df.groupby(['Transaction Type','Industry']).size().reset_index(name='count')
top_5=common_combinations.sort_values(by='count',ascending=False)
print(top_5.head(7))

     Transaction Type      Industry  count
27  Property Purchase   Real Estate    324
17  Offshore Transfer       Finance    319
2     Cash Withdrawal  Construction    318
23  Property Purchase  Construction    311
21  Property Purchase    Arms Trade    303
18  Offshore Transfer  Luxury Goods    301
4     Cash Withdrawal  Luxury Goods    301


In [106]:
#compute the mean,median and std for amount (usd) for each country
#which countries have he highest average transacton
country_stat=df.groupby('Country')['Amount (USD)'].agg(['mean','median','std']).reset_index()
country_stat_sorted= country_stat.sort_values(by='mean',ascending = False)
country_stat_sorted

,Country,mean,median,std
1,China,2.593129e+06,2.644712e+06,1.436594e+06
6,Switzerland,2.517068e+06,2.592528e+06,1.428701e+06
5,South Africa,2.510831e+06,2.455383e+06,1.398297e+06
4,Singapore,2.508388e+06,2.561971e+06,1.422815e+06
3,Russia,2.501651e+06,2.478250e+06,1.450927e+06
0,Brazil,2.498370e+06,2.465548e+06,1.423476e+06
8,UK,2.492823e+06,2.489680e+06,1.442969e+06
2,India,2.492579e+06,2.518758e+06,1.411817e+06
7,UAE,2.490049e+06,2.442782e+06,1.435933e+06
9,USA,2.407059e+06,2.362195e+06,1.390338e+06


In [107]:
#CAL THR AVERAGE MONEY LAUNDARING RISK SCORE FOR EACH COUNTRY AND COMPARE THEM.
# #ARE THERE ANY COUNTRIES THAT HAVE SIGNIFICANTLY HIGHEIR SCORES THAN OTHERS
average_risk_score=df.groupby('Country')['Money Laundering Risk Score'].mean().reset_index()
outcome=average_risk_score.sort_values('Money Laundering Risk Score',ascending=False)
outcome


,Country,Money Laundering Risk Score
2,India,5.613131
4,Singapore,5.585930
0,Brazil,5.579523
6,Switzerland,5.557360
8,UK,5.542406
1,China,5.510597
9,USA,5.492292
7,UAE,5.478484
5,South Africa,5.476608
3,Russia,5.428285


In [108]:
#ANALYSE THE PERCENTAGE OF TRANSACTIONS THAT WHERE REPORETED BY AUTHORITY AND COMPARE THE
#  #AVERAGE AMOUNT(USD) AND RISK SCORE FOR REPORTED VS NON REPORTE
total_transaction=len(df)
reported_trans=len(df[df['Reported by Authority']==1])
percentage_reported=(reported_trans/total_transaction) *100
reported_stat=df.groupby('Reported by Authority').agg({'Amount (USD)':'mean','Money Laundering Risk Score':'mean'}).reset_index()
print(f'percentage of transaction reported by authority :{percentage_reported:.2f}%')
print(reported_stat)

percentage of transaction reported by authority :20.05%
   Reported by Authority  Amount (USD)  Money Laundering Risk Score
0                  False  2.498099e+06                     5.521326
1                   True  2.516646e+06                     5.546633


In [109]:
reported_stat=df.groupby('Reported by Authority')[['Amount (USD)','Money Laundering Risk Score']].mean().reset_index()
reported_stat

,Reported by Authority,Amount (USD),Money Laundering Risk Score
0,False,2.498099e+06,5.521326
1,True,2.516646e+06,5.546633


In [110]:
#method 2
total_transaction=len(df)
not_reported_trans=len(df[df['Reported by Authority']==0])
percentage_not_reported=(not_reported_trans/total_transaction) *100
print(f'percentage of transaction not reported by authority :{percentage_not_reported:.2f}%')


percentage of transaction not reported by authority :79.95%


assignment


In [111]:
#What are the top 5 most frequent Industries involved in transactions? Compare their average Amount (USD) and Risk Score.
#21. For each Transaction Type, compute the average and median Amount (USD). Which transaction types involve higher monetary amounts?
#22. Compute the proportion of transactions marked as involving Illegal sources of money. How does this proportion vary across Countries?
#23. Compare the number of transactions and the average Risk Score for transactions involving 0, 1, or more than 1 Shell Companies.
#24. Analyze the average and median Amount (USD) for transactions involving Tax Haven Countries compared to those that do not involve tax havens. How significant is the difference?
#25. Perform a correlation analysis between Amount (USD), Money Laundering Risk Score, and Shell Companies Involved. What are the correlation coefficients, and what do they imply?


#What are the top 5 most frequent Industries involved in transactions?
#  #Compare their average Amount (USD) and Risk Score


In [112]:

df.columns





Index(['Transaction ID', 'Country', 'Amount (USD)', 'Transaction Type',
       'Date of Transaction', 'Person Involved', 'Industry',
       'Destination Country', 'Reported by Authority', 'Source of Money',
       'Money Laundering Risk Score', 'Shell Companies Involved',
       'Financial Institution', 'Tax Haven Country', 'Year_of_Transaction',
       'Month_of_Transaction', 'Day_of_Transaction', 'AMOUNT CATEGORY',
       'Time Diff', 'RISK INDEX', 'High Risk Flag'],
      dtype='object')

In [113]:
#What are the top 5 most frequent Industries involved in transactions?
top5_most_freq=df['Industry'].value_counts().sort_values(ascending=False).reset_index()
top5_most_freq



,Industry,count
0,Finance,1475
1,Construction,1460
2,Luxury Goods,1459
3,Real Estate,1443
4,Arms Trade,1414
5,Casinos,1377
6,Oil & Gas,1372


In [114]:

#Compare their average Amount (USD) and Risk Score
#df.groupby('Industry')[['Amount (USD)','Money Laundering Risk Score']].
df.groupby('Industry')[['Amount (USD)','Money Laundering Risk Score']].mean().reset_index()


,Industry,Amount (USD),Money Laundering Risk Score
0,Arms Trade,2.546362e+06,5.570721
1,Casinos,2.482550e+06,5.556282
2,Construction,2.538177e+06,5.583562
3,Finance,2.532829e+06,5.711864
4,Luxury Goods,2.465999e+06,5.373544
5,Oil & Gas,2.466224e+06,5.427843
6,Real Estate,2.478126e+06,5.455301


In [115]:
#21. For each Transaction Type, compute the average and median Amount (USD). 
# #Which transaction types involve higher monetary amounts?T

In [116]:
df.groupby('Transaction Type')[['Amount (USD)']].agg(['mean','median']).reset_index()


Transaction Type  Amount (USD)              
                              mean        median
0    Cash Withdrawal  2.507551e+06  2.524111e+06
1     Cryptocurrency  2.521129e+06  2.551082e+06
2  Offshore Transfer  2.527693e+06  2.515103e+06
3  Property Purchase  2.450534e+06  2.374724e+06
4    Stocks Transfer  2.504995e+06  2.546577e+06

In [117]:
#22. Compute the proportion of transactions marked as involving Illegal sources of money.#
#  How does this proportion vary across Countries?

In [118]:
df[['Source of Money']].value_counts()

Source of Money
Illegal            7017
Legal              2983
Name: count, dtype: int64

In [119]:
#  How does this proportion vary across Countries?
df.groupby('Source of Money')[['Country']].value_counts().reset_index()

,Source of Money,Country,count
0,Illegal,Brazil,748
1,Illegal,South Africa,728
2,Illegal,China,711
3,Illegal,UK,708
4,Illegal,Russia,701
5,Illegal,India,695
6,Illegal,Switzerland,687
7,Illegal,Singapore,685
8,Illegal,UAE,685
9,Illegal,USA,669


In [120]:
#23. Compare the number of transactions and the average Risk Score for transactions 
# #involving 0, 1, or more than 1 Shell Companies.


In [121]:
df.groupby('Shell Companies Involved')[['Money Laundering Risk Score']].mean().reset_index()

,Shell Companies Involved,Money Laundering Risk Score
0,0,5.619545
1,1,5.698990
2,2,5.555000
3,3,5.482125
4,4,5.359799
5,5,5.586006
6,6,5.470352
7,7,5.587576
8,8,5.572008
9,9,5.325253


In [122]:
#24. Analyze the average and median Amount (USD) for transactions involving 
# #Tax Haven Countries compared to those that do not involve tax havens. 
# #How significant is the difference?


In [123]:
df.groupby('Tax Haven Country')[['Amount (USD)']].agg(['mean','median']).reset_index()

Tax Haven Country  Amount (USD)              
                             mean        median
0           Bahamas  2.471600e+06  2.455664e+06
1    Cayman Islands  2.502189e+06  2.515616e+06
2        Luxembourg  2.502509e+06  2.468006e+06
3            Panama  2.500342e+06  2.552703e+06
4         Singapore  2.510816e+06  2.515120e+06
5       Switzerland  2.523432e+06  2.496544e+06

In [124]:
#25. Perform a correlation analysis between Amount (USD), Money Laundering Risk Score, and Shell Companies Involved. What are the correlation coefficients, and what do they imply?

In [125]:
df.groupby('Amount (USD)')[['Money Laundering Risk Score','Shell Companies Involved']].corr().reset_index()



,Amount (USD),level_1,Money Laundering Risk Score,Shell Companies Involved
0,1.003180e+04,Money Laundering Risk Score,NaN,NaN
1,1.003180e+04,Shell Companies Involved,NaN,NaN
2,1.025862e+04,Money Laundering Risk Score,NaN,NaN
3,1.025862e+04,Shell Companies Involved,NaN,NaN
4,1.053049e+04,Money Laundering Risk Score,NaN,NaN
...,...,...,...,...
19995,4.999336e+06,Shell Companies Involved,NaN,NaN
19996,4.999461e+06,Money Laundering Risk Score,NaN,NaN
19997,4.999461e+06,Shell Companies Involved,NaN,NaN
19998,4.999812e+06,Money Laundering Risk Score,NaN,NaN
